In [1]:
import tensorflow as tf
from tensorflow.keras import Sequential, backend as K
from tensorflow.keras.layers import Dense, LSTM, Dropout
import pandas as pd
import os
import numpy as np
import sklearn
from sklearn.model_selection import train_test_split
#from tensorflow.keras import backend as K
#from keras.activations import softmax
from tensorflow.keras.losses import categorical_crossentropy

DIR = '/home/arun/Models/'
train = os.path.join(DIR, 'train.csv')
test = os.path.join(DIR, 'test.csv')
valid = os.path.join(DIR, 'validate.csv')

START = 8
TIMERANGE = 20
TIMERANGE_TEST = 12
TEST_START = START + TIMERANGE + 1

CATCOLS = ['occupancy_status', 'channel', 'property_state', 'property_type', 
           'loan_purpose', 'mortgage_insurance_type', 'servicer_default_grp',
           'servicer_prepay_grp', 'vintage_yr', 'dq_status']

CATCOLS = ['occupancy_status', 'channel', 'property_type']

pd.set_option('display.max_rows', 500)

In [2]:
skip_cols = ['validation_flag','test_train']
#testdata = pd.read_csv(test, index_col=False, nrows=(1*10**5),usecols = lambda column:column not in 
#skip_cols)
traindata = pd.read_csv(train, index_col=False, nrows=(1*10**6),usecols = lambda column:column not in 
skip_cols)
validata = pd.read_csv(valid, index_col=False, nrows=(4*10**5),usecols = lambda column:column not in 
skip_cols)
hist = traindata #pd.concat([traindata,testdata,validata]) 
hist = pd.get_dummies(hist, columns=CATCOLS)

In [5]:
test_real, dev_real = sklearn.model_selection.train_test_split(validata, test_size = 0.5, train_size = 0.5)

In [6]:
testdata = pd.get_dummies(test_real,columns=CATCOLS)
devdata = pd.get_dummies(dev_real,columns=CATCOLS)

In [ ]:
#traindata.prepay_flg.values.sum()
testdata.default_flg.values.sum()

In [ ]:
#validata.prepay_flg.values.sum()
devdata.default_flg.values.sum()

In [ ]:
traindata.groupby(['mortgage_insurance_type']).count()['loan_sequence_number']

In [ ]:
validata.groupby(['mortgage_insurance_type']).count()['loan_sequence_number']

In [ ]:
hist.columns

In [7]:
def generate_XY(hist):
    X = None
    Y = None
    data = np.concatenate((np.arange(START, START + TIMERANGE).reshape(-1,1), np.ones((TIMERANGE,1))), axis=1)
    dummy_df = pd.DataFrame(data, columns=['loan_age', 'removed'])
    lsn = []

    for loan_num, group_df in hist.groupby(['loan_sequence_number']):
        # Clean up instances where default flag is raise but loan persists
        if group_df[group_df['default_flg'] == 1].shape[0] > 1:
            first_idx = group_df[group_df['default_flg'] == 1]['default_flg'].index[0]
            group_df = group_df.loc[:first_idx,:]

        tmpY = group_df[['prepay_flg', 'default_flg', 'loan_age']]
        
        if not tmpY.iloc[:TIMERANGE,[0,1]].any().any():
            lastYVal = tmpY[TIMERANGE:TIMERANGE+12].any()[0:2] * 1
            
            
        
        tmpY['current'] = True
        tmpY['current'].iloc[-1] = ~tmpY['prepay_flg'].astype(bool).iloc[-1] & ~tmpY['default_flg'].astype(bool).iloc[-1]
        tmpY = dummy_df.merge(tmpY, on='loan_age', how='left').fillna(0)
        if not tmpY.iloc[:,[2,3]].any().any():
            tmpY.iloc[TIMERANGE-1,[2,3]] = lastYVal
        tmpY = tmpY.drop(['loan_age'], axis=1)
        tmpY = tmpY.astype(bool)
        tmpY.loc[:,'removed'] = ~tmpY['current'] & ~tmpY['prepay_flg'] & ~tmpY['default_flg']
        tmpY = np.array(tmpY).reshape((1, tmpY.shape[0], tmpY.shape[1]))

        lsn.append(loan_num)
        tmpX = group_df.drop(['loan_sequence_number', 'prepay_flg', 'default_flg'], axis=1)
        tmpX = dummy_df.merge(tmpX, on='loan_age', how='left').fillna(0)
        tmpX = np.array(tmpX).reshape((1, tmpX.shape[0], tmpX.shape[1]))

        if X is not None:
            X = np.append(X, tmpX, axis=0)
            Y = np.append(Y, tmpY, axis=0)
        else:
            X = tmpX
            Y = tmpY
    return (X,Y,lsn)

In [9]:
X,Y, lsn = generate_XY(hist)

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-cop

In [1]:
import pickle

In [30]:
######This is only for writing....No need to run everytime#######
with open('devdata.pkl', 'wb') as f:
    pickle.dump((Xdev,Ydev,lsntest), f)

In [2]:
######This is only for reading....No need to run everytime#######

with open('traindata.pkl', 'rb') as f: 
    X2, Y2, lsn2 = pickle.load(f)

In [28]:
Xtest.shape

(29702, 20, 45)

In [27]:
Xtest,Ytest, lsntest = generate_XY(testdata)

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-cop

In [10]:
Xdev,Ydev, lsndev = generate_XY(devdata)

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [26]:
Xdev.shape

(29704, 20, 45)

In [ ]:
for loan_num, group_df in hist.groupby(['loan_sequence_number']):

        if group_df[group_df['default_flg'] == 1].shape[0] > 1:
            first_idx = group_df[group_df['default_flg'] == 1]['default_flg'].index[0]
            group_df = group_df.loc[:first_idx,:]

        if group_df.shape[0] > 10:
            tmpY = group_df[['prepay_flg', 'default_flg', 'loan_age']]

In [31]:
def weighted_categorical_crossentropy(weights):
    """
    A weighted version of keras.objectives.categorical_crossentropy
    
    Variables:
        weights: numpy array of shape (C,) where C is the number of classes
    
    Usage:
        weights = np.array([0.5,2,10]) # Class one at 0.5, class 2 twice the normal weights, class 3 10x.
        loss = weighted_categorical_crossentropy(weights)
        model.compile(loss=loss,optimizer='adam')
    """
    
    weights = K.variable(weights)
        
    def loss(y_true, y_pred):
        # scale predictions so that the class probas of each sample sum to 1
        y_pred /= K.sum(y_pred, axis=-1, keepdims=True)
        # clip to prevent NaN's and Inf's
        y_pred = K.clip(y_pred, K.epsilon(), 1 - K.epsilon())
        # calc
        loss = y_true * K.log(y_pred) * weights
        loss = -K.sum(loss, -1)
        return loss
    
    return loss

In [ ]:
#hyperparameter tuning
hiddensizes = [15,20, 30, 40]
dropouts = [0,0.8,0.15,0.2]
batch_sizes = [32,64,128]
optimizers = ['rmsprop', 'adam']
for dropout in dropouts:
    for batch_size in batch_sizes:
        for optimizer in optimizers:
            for hidden in hiddensizes:
                print('dropout:' + str(dropout) + '\t' + 'batchsize:' + str(batch_size) + '\t' + 'optimzer:' + str(optimizer)  + '\t' + 'Hidden_U:' + str(hidden))

                regressor = Sequential()
                #hiddensize1 = X.shape[2]

                regressor.add(LSTM(units = hidden, dropout=dropout, return_sequences = True, input_shape = (X.shape[1], X.shape[2])))
                regressor.add(LSTM(units = hidden, dropout=dropout, return_sequences = True))
                #regressor.add(LSTM(units = hiddensize1, dropout=0.2, return_sequences = True))

                regressor.add(Dense(units = 4, activation='softmax'))
                weights = np.array([0.005,0.24,0.75,0.005])
                #weights = np.ones((4,))*1.0
                regressor.compile(optimizer = optimizer, loss = weighted_categorical_crossentropy(weights), metrics = ['categorical_accuracy'])
                regressor.fit(X, Y*1.0,batch_size=batch_size, epochs=4, verbose=1, validation_data=(Xtest, Ytest*1.0))
        

dropout:0.15	batchsize:32	optimzer:rmsprop	Hidden_U:30
Train on 40521 samples, validate on 29693 samples
Epoch 1/4
40521/40521 [==============================] - 50s 1ms/sample - loss: 0.0071 - categorical_accuracy: 0.5704 - val_loss: 0.0043 - val_categorical_accuracy: 0.8727
Epoch 2/4
40521/40521 [==============================] - 46s 1ms/sample - loss: 0.0046 - categorical_accuracy: 0.8513 - val_loss: 0.0050 - val_categorical_accuracy: 0.8272
Epoch 3/4
40521/40521 [==============================] - 42s 1ms/sample - loss: 0.0040 - categorical_accuracy: 0.8793 - val_loss: 0.0037 - val_categorical_accuracy: 0.8616
Epoch 4/4
40521/40521 [==============================] - 43s 1ms/sample - loss: 0.0038 - categorical_accuracy: 0.8943 - val_loss: 0.0037 - val_categorical_accuracy: 0.8543
dropout:0.15	batchsize:32	optimzer:rmsprop	Hidden_U:40
Train on 40521 samples, validate on 29693 samples
Epoch 1/4
40521/40521 [==============================] - 64s 2ms/sample - loss: 0.0060 - categorical_a

In [38]:
print('dropout:0.08'  + '\t' + 'batchsize:64' +  '\t' + 'optimzer: RMSprop' + '\t' + 'Hidden_U:20' )

regressor = Sequential()
#hiddensize1 = X.shape[2]

regressor.add(LSTM(units = 20, dropout=0.08, return_sequences = True, input_shape = (X.shape[1], X.shape[2])))
regressor.add(LSTM(units = 20, dropout=0.08, return_sequences = True))
#regressor.add(LSTM(units = hiddensize1, dropout=0.2, return_sequences = True))

regressor.add(Dense(units = 4, activation='softmax'))
weights = np.array([0.005,0.24,0.75,0.005])
#weights = np.ones((4,))*1.0
regressor.compile(optimizer = 'rmsprop', loss = weighted_categorical_crossentropy(weights), metrics = ['categorical_accuracy'])
regressor.fit(X, Y*1.0,batch_size=64, epochs=10, verbose=1, validation_data=(Xdev, Ydev*1.0))
        

dropout:0.08	batchsize:64	optimzer: RMSprop	Hidden_U:20
Train on 40521 samples, validate on 29704 samples
Epoch 1/10
40521/40521 [==============================] - 29s 717us/sample - loss: 0.0065 - categorical_accuracy: 0.6945 - val_loss: 0.0030 - val_categorical_accuracy: 0.8894
Epoch 2/10
40521/40521 [==============================] - 26s 636us/sample - loss: 0.0041 - categorical_accuracy: 0.8872 - val_loss: 0.0029 - val_categorical_accuracy: 0.9033
Epoch 3/10
40521/40521 [==============================] - 27s 672us/sample - loss: 0.0037 - categorical_accuracy: 0.9179 - val_loss: 0.0018 - val_categorical_accuracy: 0.9258
Epoch 4/10
40521/40521 [==============================] - 27s 671us/sample - loss: 0.0034 - categorical_accuracy: 0.9334 - val_loss: 0.0015 - val_categorical_accuracy: 0.9387
Epoch 5/10
40521/40521 [==============================] - 29s 715us/sample - loss: 0.0031 - categorical_accuracy: 0.9430 - val_loss: 0.0012 - val_categorical_accuracy: 0.9508
Epoch 6/10
40521/40

In [45]:
regressor.save('regressor.h5', save_format='tf')

In [49]:
regressor2 = tf.keras.models.load_model('regressor.h5', custom_objects={'loss':weighted_categorical_crossentropy(weights)})

In [ ]:
Ypred = regressor.predict(X)

In [ ]:
X.shape

In [ ]:
Xtest,Ytest, lsntest = generate_XY(testdata)

In [ ]:
len(lsn)

In [ ]:
YpredTest = regressor.predict(Xtest)

In [ ]:
Xdev,Ydev, lsndev = generate_XY(validata)

In [ ]:
idx=2272
Ypred[idx]

In [ ]:
hist[hist.loan_sequence_number==lsn[idx]][['prepay_flg','default_flg','loan_age']]

In [ ]:
hist[hist.default_flg==1]['loan_sequence_number']

In [ ]:
np.where(np.array(lsn) == 668620845349)